In [1]:
import torch

# Check if MPS (Metal Performance Shaders) is available and set the device accordingly
device = torch.device("mps" if torch.backends.mps.is_built() else "cpu")
print("Using device:", device)

Using device: mps


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

prompt = "A yellow man is "
inputs = tokenizer(prompt, return_tensors="pt")

# Move inputs to the appropriate device
model.to(device)
inputs.to(device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=30,
        num_return_sequences=1,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

A yellow man is iced. I can't see his face, as I can't see his eyes.

"Hear this from my


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

input_text = """
The Apollo program was a series of space missions undertaken by the United States with the goal of landing humans on the Moon and bringing them back safely. It was initiated in the early 1960s and culminated with the Apollo 11 mission in 1969.
"""
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

# Move inputs to the appropriate device
model.to(device)
inputs.to(device)

with torch.no_grad():
    summary_ids = model.generate(
        **inputs,
        max_length=50,
        min_length=15,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )


summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


The Apollo program was a series of space missions undertaken by the United States with the goal of landing humans on the Moon and bringing them back safely. It was initiated in the early 1960s and culminated with the Apollo 11 mission in 1969.


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

model_id = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

text = "I like to eat shit"
inputs = tokenizer(text, return_tensors="pt")

# Move inputs to the appropriate device
model.to(device)
inputs.to(device)

with torch.no_grad():
    logits = model(**inputs).logits
    probs = F.softmax(logits, dim=-1)

label = torch.argmax(probs).item()
confidence = probs[0][label].item()
print(f"Label: {label} (Confidence: {confidence:.2f})")


Label: 0 (Confidence: 1.00)
